In [ ]:
import os
import sys
from google.colab import drive 
# Enlazar a la carpeta 'data'
drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/Colab Notebooks/ICBF/data')
sys.path.insert(0, '../scripts/0_utils')
!pwd

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Colab Notebooks/ICBF/data


# **Implementación de los modelos predictivos**

####**Descripción:**
Define funciones para hallar la probabilidad de que un beneficiario se encuentre en situación de desnutrición después de uno o dos meses de su última valoración nutricional, a partir de los modelos predictivos construidos, usando los z-scores de las dos tomas nutricionales más recientes como predictores.
 
####**Requerimientos:**

- Modelos serializados `models/model_min_i.pickle`, con `i` = 1, 3, que predice si un beneficiario que ha sufrido de desnutrición pero no está desnutrido en la actualidad recaerá después de 1 o 3 meses, en función de sus z-scores peso/talla, peso/edad e IMC actuales e inmediatamento anteriores. 
- Lista serializada `models/model_min_1_cols.pickle`, de las variables nutricionales utilizadas por ambos modelos como predictores.
- Tabla de datos nutricionales limpia `clean_data/tomas_nutricionales.parquet` para probar las funciones de predicción.

####**Salidas:**

Ninguna. Define las siguientes funciones:
- `predict_set(dataset, model, col_model, depth=2)` que retorna una tabla de relación entre el identificador `IdBeneficiario` de cada beneficiario y su probabilidad de sufrir desnutrición (*p-score*) en el futuro, de acuerdo  al conjunto de datos y al modelo ingresado.
- `predict_report(dataset)` que adjunta a la tabla de datos ingresada los valores de probabilidad de sufrir desnutricion en un plazo de 1 y 3 meses, como nuevas columnas.
- `predict_IdB(data, IdBeneficiario, model, col_model, depth=2)` que retorna la probabilidad de sufrir desnutrición de un usuario específico, de acuerdo al conjunto de datos y al modelo ingresado.

Todas las funciones requieren la librería `re` para su correcto funcionamiento.

In [ ]:
# Librerias relevantes
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import re

# Opciones de visualización
pd.options.display.max_columns = 100
pd.options.display.max_rows = 100
pd.options.mode.chained_assignment = None

In [ ]:
# Abre el dataframe de todas las tomas
tom = pd.read_parquet('clean_data/tomas_nutricionales.parquet')
# Extrae una muestra de beneficiarios
IdB_sample = tom['IdBeneficiario'].drop_duplicates().sample(1000, random_state=12)
sample = tom[tom['IdBeneficiario'].isin(IdB_sample)]
# Carga los modelos mínimos de predicción a partir de tomas anterior y trasanterior
clf_1 = joblib.load('models/model_min_1.pickle')
clf_3 = joblib.load('models/model_min_3.pickle')
# Columnas usadas por los modelos como predictores
cols_clf = joblib.load('models/model_min_1_cols.pickle')

### **Función de predicción a partir de un conjunto de datos nutricionales**

In [ ]:
def predict_set(dataset, model, col_model, depth=2):
  ''' 
  Calcula la probabilidad de recurrencia en desnutrición de los beneficiarios de
  un tabla de datos 'dataset' que tengan 'depth' o más tomas nutricionales, de acuerdo
  al modelo predictivo 'model', utilizando las columnas 'col_model' como predictores.
  Retorna una tabla de relación entre cada identificador 'IdBeneficiario' y su respectivo
  'p_score' (probabilidad de sufrir desnutrición).
  import pandas as pd
  import numpy as np
  import re
  '''
  dias_mes = 365.25 / 12
  df = dataset.sort_values(['IdBeneficiario', 'EdadMeses'])
  cols_imp = ['IdBeneficiario'] + list({re.sub('-\d$', '', col) for col in col_model})
  df = df.dropna(subset=cols_imp).reset_index(drop=True)
  if len(df) > 0:
    his = df[cols_imp]
    # Para cada periodo tomado en cuenta (de 1 a 'depth')
    for i in range(depth):
      periodo = i + 1
      # Coloca al frente de cada fila, los datos de la fila que está 'periodo' filas antes
      for col in cols_imp:
        his[f'{col}-{periodo}'] = his[col].shift(periodo)
    # Conserva solo las filas donde todos los datos pertenecen al mismo beneficiario
    his = his.loc[his['IdBeneficiario'] == his[f'IdBeneficiario-{depth}']]
    # Calcula el tiempo desde la última toma redondeado al número de meses
    his['tiempo'] = (his['EdadMeses'] - his['EdadMeses-1']).round(0)
    # Deja solo las columnas de la predicción y el identificador del beneficiario
    his = his[['IdBeneficiario'] + col_model].\
      drop_duplicates(subset=['IdBeneficiario'], keep='last')
    IdBeneficiario = his['IdBeneficiario']
    X = his.drop(columns=['IdBeneficiario'])
    y = [q for p, q in model.predict_proba(X)]
    prediction = pd.DataFrame({'IdBeneficiario': IdBeneficiario, 'p_score': y})
    prediction = prediction.drop_duplicates(subset=['IdBeneficiario'], keep='last')
    return prediction
  else:
    print('Datos insuficientes.')
    return np.nan
# Prueba la función de predicción en la muestra
prediction = predict_set(sample, clf_1, cols_clf, depth=2)
prediction

IdBeneficiario   p_score
5            3294599  0.757277
9            3359657  0.256938
12           3571003  0.237300
17           4847823  0.218549
21           4860609  0.727879
...              ...       ...
5657        18562113  0.290360
5662        18565747  0.879304
5666        18588966  0.481430
5694        18722451  0.361005
5698        18724331  0.255794

[809 rows x 2 columns]

### **Función de generación de reporte con predicciones**

In [ ]:
def predict_report(dataset):
  ''' 
  Calcula la probabilidad de recurrencia en desnutrición de los beneficiarios de
  un tabla de datos 'dataset' que tengan 2 o más tomas nutricionales, para 1 y 3 meses
  de acuerdo a los modelos 'model_min_1' y 'model_min_3', respectivamente,  que utilizan
  los valores de z-score peso/talla, peso/edad e IMC de las dos tomas nutricionales 
  anteriores, como predictores.
  Retorna una tabla de datos similar a la ingresada con dos nuevas columnas que corresponden
  al 'p_score' de cada individuo (probabilidad de sufrir desnutrición).
  import pandas as pd
  import numpy as np
  import re
  import predict_set
  '''
  # Predicción a un mes
  prediction_1 = predict_set(dataset, clf_1, cols_clf, depth=2).\
    rename(columns={'p_score': 'riesgo_desnutricion_1_mes'})
  # Predicción a tres meses
  prediction_3 = predict_set(dataset, clf_3, cols_clf, depth=2).\
    rename(columns={'p_score': 'riesgo_desnutricion_3_meses'})
  # Unión de las predicciones con la tabla de entrada
  # conservando solo la última valoración nutricional de cada beneficiario
  predictions = pd.merge(prediction_1, prediction_3, on='IdBeneficiario', how='outer') 
  return dataset.drop_duplicates(subset=['IdBeneficiario'], keep='last').\
    merge(predictions, on='IdBeneficiario', how='left')
# Prueba la función de predicción en la muestra
report = predict_report(sample)
report

IdToma  Registro  Vigencia Toma  \
0        3446   1001578      2017    4   
1        9558   1040116      2017    4   
2       10904   1051738      2017    4   
3       20059   1106281      2017    4   
4       31359   1175644      2017    2   
..        ...       ...       ...  ...   
995  19264040   5250241      2019    4   
996  19265550   5290312      2019    4   
997  19267258   5336676      2019    4   
998  19267351   5338941      2019    4   
999  19268366   4669355      2018   2N   

                                              Servicio  \
0           CDI SIN ARRIENDO -  INSTITUCIONAL INTEGRAL   
1                     HCB TRADICIONAL- COMUNITARIO (T)   
2          HOGARES INFANTILES - INSTITUCIONAL INTEGRAL   
3    SERVICIO ESPECIAL PARA LA PRIMERA INFANCIA - F...   
4           CDI SIN ARRIENDO -  INSTITUCIONAL INTEGRAL   
..                                                 ...   
995  SERVICIO ESPECIAL PARA LA PRIMERA INFANCIA - F...   
996  DESARROLLO INFANTIL EN MEDIO FAMILIAR SIN ARRI...   
997  DESARROLLO INFANTIL EN MEDIO FAMILIAR SIN ARRI...   
998                      HCB FAMI-FAMILIAR TRADICIONAL   
999  ESTRATEGIAS DE DESARROLLO ALIMENTARIO O NUTRIC...   

    FechaValoracionNutricional  EdadMeses FechaMedicionPerimetroBraquial  \
0                   2017-10-18       51.0                            NaT   
1                   2017-11-09       62.0                            NaT   
2                   2017-11-21       61.0                            NaT   
3                   2017-11-08       60.0                            NaT   
4                   2017-05-08       53.0                            NaT   
..                         ...        ...                            ...   
995                 2019-12-05       26.0                     2019-12-05   
996                 2019-12-10        0.0                            NaT   
997                 2019-12-02        0.0                            NaT   
998                 2019-12-04        1.0                            NaT   
999                 2019-12-10       23.0                     2019-11-20   

     MedicionPerimetroBraquial  Peso  Talla  ZScoreTallaEdad  ZScorePesoEdad  \
0                          NaN  14.7   99.0            -1.29           -0.92   
1                          NaN  14.0  101.0            -1.90            0.00   
2                          NaN  20.1  112.0             0.38            0.00   
3                          NaN  18.5  107.0            -0.54            0.00   
4                          NaN  16.5  107.0             0.28           -0.22   
..                         ...   ...    ...              ...             ...   
995                       13.0  12.0   88.0             0.14            0.06   
996                        NaN   3.5   51.0            -1.24           -1.06   
997                        NaN   3.6   51.0            -0.83           -0.52   
998                        NaN   4.7   56.0            -0.06           -0.31   
999                       13.4   9.0   78.0            -2.57           -2.01   

     ZScorePesoTalla  ZScoreIMC              EstadoTallaEdad  \
0              -0.18      -0.18         Riesgo de baja talla   
1              -1.15      -1.13         Riesgo de baja talla   
2               0.48       0.60  Talla adecuada para la edad   
3               0.60       0.57  Talla adecuada para la edad   
4              -0.62      -0.61  Talla adecuada para la edad   
..               ...        ...                          ...   
995            -0.14      -0.10  Talla adecuada para la edad   
996            -0.12      -0.60         Riesgo de baja talla   
997             0.11      -0.13  Talla adecuada para la edad   
998            -0.33      -0.42  Talla adecuada para la edad   
999            -0.85       0.00             Retraso en talla   

                       EstadoPesoEdad               EstadoPesoTalla  \
0          Peso adecuado para la edad   Peso adecuado para la talla   
1          Peso adec

### **Función de predicción para un único beneficiario a partir de un conjunto de datos nutricionales**

In [ ]:
def predict_IdB(data, IdBeneficiario, model, col_model, depth=2):
  ''' 
  Calcula la probabilidad de recurrencia en desnutrición de un beneficiario, a partir
  de su identificador 'IdBeneficiario' y sus registros en un tabla de datos 'dataset' 
  que tengan 'depth' o más tomas nutricionales del beneficiario, de acuerdo al modelo
  predictivo 'model', utilizando las columnas 'col_model' como predictores.
  Retorna el respectivo 'p_score' (probabilidad de sufrir desnutrición) asociado al
  beneficiario.
  De no ser posible realizar la predicción, retorna un NaN.
  import pandas as pd
  import numpy as np
  import re
  import predict_set
  '''
  # Registros del beneficiario
  subset = data[data['IdBeneficiario'] == IdBeneficiario]
  try:
    # Función de predicción
    return predict_set(subset, model, col_model, depth=depth)['p_score'].iloc[0]
  except:
    # Si no es posible hallar el p-score, retorna un NaN
    print('Datos insuficientes.')
    return np.nan
# Prueba la función de predicción a partir del identificador 'IdBeneficiario'
predict_IdB(tom, 13822741, clf_1, cols_clf)

0.23049190483005189